In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from PIL import Image
import os

In [2]:
#import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Instructions for updating:
non-resource variables are not supported in the long term


Should take around 3 minutes to load

In [3]:
h_dim = 1000
v_dim = 750
print("Loading Data from Memory")

root = "Train"
label_flood_dir = os.path.join(root,'Labeled','Flooded','image')
label_nonflood_dir = os.path.join(root,'Labeled','Non-Flooded','image')
flooded_img = []
nonflooded_img = []

for file in os.listdir(label_flood_dir):
    image = Image.open(os.path.join(label_flood_dir, file))
    flooded_img.append(np.array(image.resize((h_dim,v_dim))))
    
for file in os.listdir(label_nonflood_dir):
    image = Image.open(os.path.join(label_nonflood_dir, file))
    nonflooded_img.append(np.array(image.resize((h_dim,v_dim))))
print("Loaded!")

Loading Data from Memory
Loaded!


In [4]:
print("Flooded Image Shape: {}".format(flooded_img[0].shape))
print("Non_Flooded Image Shape: {}".format(nonflooded_img[0].shape))

data_img = np.vstack((np.array(flooded_img), np.array(nonflooded_img))) / 255.
data_img.shape

Flooded Image Shape: (750, 1000, 3)
Non_Flooded Image Shape: (750, 1000, 3)


(398, 750, 1000, 3)

In [5]:
from utils import train_test_split
train_idx, test_idx, train_labels, test_labels = train_test_split(flooded_img, nonflooded_img)

Data len 51; n is 51 ;train size 40, test size 11
Data len 347; n is 347 ;train size 277, test size 70
Training Indices len 317
Testing Indices len 81


In [6]:
# convolutional layer
# x the input 
# shape is dimension of input
def conv_layer(x, shape,stride):
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

    bias = tf.Variable(tf.constant(0.05, shape=[shape[-1]]))

    out = tf.nn.conv2d(input=x, filters=weights, strides=stride, padding='VALID')
    out += bias
    return out

# pooling layer
def max_pool(x, k=2):

    out = tf.nn.max_pool(value=x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='VALID')
    return out

# fully connected layer
def fully_connected_layer(x, shape):
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

    bias = tf.Variable(tf.constant(0.05, shape=[shape[1]]))

    out = tf.matmul(a=x, b=weights)
    out += bias
    return out

# flatten layer
def flatten_layer(x):
    
    size = x.get_shape()[1:4].num_elements()
    out = tf.reshape(x, [-1,size])
    return out, size

# relu
relu = lambda x: tf.nn.relu(features=x)

# softmax
softmax = lambda x: tf.nn.softmax(logits=x)

# sigmoid
sigmoid = lambda x: tf.nn.sigmoid(x)

# batch norm
batch_norm = lambda x: tf.layers.batch_normalization(x)

In [13]:
# define CNN
def toy_model(x,show_dim = False):
    
    # Six convolutional layers with max pool and ReLU
    
        #shape = [filter_size, filter_size, num_input_channels, num_filters]
    def conv_block(x,in_channels,out_channels, show_dim = False, stride = 2, kernel_h=10,kernel_w =10,):
        shape = [kernel_h,kernel_w,in_channels,out_channels]
        x = conv_layer(x, shape,stride)
        x = relu(x)
        x = batch_norm(x)
        x = max_pool(x, k=2)
        if show_dim:
            print(x.shape)
        return x
    
    x = conv_block(x,3,16,show_dim)

    x = conv_block(x,16,16,show_dim)
        
    x = conv_block(x,16,32,show_dim)
        
    #x = conv_block(x,32,32,show_dim)
    
    #x = conv_block(x,64,64,show_dim)

    # flatten output and put through a fully connected layer
    flat1, size1 = flatten_layer(x)
    print(flat1.shape)
    fc1 = fully_connected_layer(flat1, [size1, 64])
    fc1 = relu(fc1)
    if show_dim:
        print(fc1.shape)

    fc2 = fully_connected_layer(fc1, [64, 1])
    if show_dim:
        print(fc2.shape)

    return fc2

# dont pad, larger kernel size, do reduce feature space and less comp expensive

- with unbalanced training, change the loss such that the minority gets much more weight

In [14]:


# def sharpen(p):
#     T = 0.5
#     pred = p**(1./T)/(p**(1./T) + (1.-p)**(1./T))
#     return pred

In [15]:
# define inputs
x = tf.placeholder(tf.float32, [None, v_dim, h_dim, 3])
y = tf.placeholder(tf.float32, [None, 1])
y_train_true = np.array(train_labels).reshape(-1,1)
y_test_true = np.array(test_labels).reshape(-1,1)

# run model with placeholder tensors
pred = toy_model(x,show_dim = True)

# sharpen
# pred = sharpen(pred)

# define loss
cross_entropy = tf.losses.sigmoid_cross_entropy(logits=pred, multi_class_labels=y)
cost = tf.reduce_mean(cross_entropy)

# define accuracy
pred_class = tf.round(sigmoid(pred))
pred_correct = tf.equal(pred_class, tf.cast(y, tf.float32))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

# initialize variables
init = tf.global_variables_initializer()
training_iters = 450
batch_size = 16 #len(train_idx)

(?, 185, 248, 16)
(?, 44, 60, 16)
(?, 9, 13, 32)
(?, 3744)
(?, 64)
(?, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# train model
with tf.Session(config=config) as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    best_acc = 0.
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    num_batches = len(train_idx)//batch_size
    
    for i in range(training_iters):
        
        # Reset metrics
        loss_total = 0
        acc_total = 0
        train_results = []
   
        # Run optimization 
        # Calculate batch loss and accuracy
        for batch in range(num_batches):
            batch_x = data_img[train_idx,:,:,:][batch*batch_size:min((batch+1)*batch_size,len(train_idx))]
            batch_y = y_train_true[batch*batch_size:min((batch+1)*batch_size,len(y_train_true))]    

            feed_dict={x: batch_x, y: batch_y}
            opt = sess.run(optimizer, feed_dict=feed_dict)
            loss, acc, pred_labels = sess.run([cost, accuracy, pred_class], feed_dict=feed_dict)
            loss_total += loss
            acc_total += acc
            train_results.append(pred_labels)

        # Average metrics
        ave_loss = loss_total/num_batches
        ave_acc = acc_total/num_batches

        # Calculate accuracy for all test images
        valid_loss, test_acc, test_results = sess.run([cost, accuracy, pred_class],
                                feed_dict={x: data_img[test_idx,:,:,:], y : y_test_true})
        
        # Update metrics
        train_loss.append(ave_loss)
        test_loss.append(valid_loss)
        train_accuracy.append(ave_acc)
        test_accuracy.append(test_acc)
        if test_acc > best_acc:
            best_model_train_labels = tf.stack(tf.reshape(tf.stack(train_results),[-1,1])).eval()
            best_model_test_labels = test_results
            best_acc = test_acc
        
        # Print metrics
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(ave_loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(ave_acc)+ \
                      " ,Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.525604, Training Accuracy= 0.74013 ,Testing Accuracy: 0.86420
Iter 1, Loss= 1.278190, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 2, Loss= 0.809888, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 3, Loss= 0.419077, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 4, Loss= 0.682539, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 5, Loss= 0.377895, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 6, Loss= 0.417774, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 7, Loss= 0.415782, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 8, Loss= 0.416516, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 9, Loss= 0.408797, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 10, Loss= 0.408063, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 11, Loss= 0.408231, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 12, Loss= 0.407948, Training Accuracy= 0.86842 ,Testing A

Iter 104, Loss= 0.403113, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 105, Loss= 0.403107, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 106, Loss= 0.403081, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 107, Loss= 0.403075, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 108, Loss= 0.403051, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 109, Loss= 0.403038, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 110, Loss= 0.403021, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 111, Loss= 0.403006, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 112, Loss= 0.402986, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 113, Loss= 0.402979, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 114, Loss= 0.402954, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 115, Loss= 0.402943, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 116, Loss= 0.402916, Training Accur

Iter 207, Loss= 0.401334, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 208, Loss= 0.401316, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 209, Loss= 0.401299, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 210, Loss= 0.401282, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 211, Loss= 0.401265, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 212, Loss= 0.401248, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 213, Loss= 0.401231, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 214, Loss= 0.401215, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 215, Loss= 0.401198, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 216, Loss= 0.401182, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 217, Loss= 0.401165, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 218, Loss= 0.401149, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 219, Loss= 0.401133, Training Accur

Iter 310, Loss= 0.399801, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 311, Loss= 0.399854, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 312, Loss= 0.399785, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 313, Loss= 0.399723, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 314, Loss= 0.399676, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 315, Loss= 0.399652, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 316, Loss= 0.399680, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 317, Loss= 0.399818, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 318, Loss= 0.399949, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 319, Loss= 0.399769, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 320, Loss= 0.399662, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 321, Loss= 0.399813, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 322, Loss= 0.399862, Training Accur

Iter 413, Loss= 0.398808, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 414, Loss= 0.398750, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 415, Loss= 0.398647, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 416, Loss= 0.398623, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 417, Loss= 0.398709, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 418, Loss= 0.398808, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 419, Loss= 0.398730, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 420, Loss= 0.398574, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 421, Loss= 0.398508, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 422, Loss= 0.398467, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 423, Loss= 0.398435, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 424, Loss= 0.398423, Training Accuracy= 0.86842 ,Testing Accuracy: 0.86420
Iter 425, Loss= 0.398427, Training Accur

In [ ]:
best_model_train_labels

In [ ]:
best_model_test_labels

In [ ]:
for title, data in {"Loss":train_loss,"Train Accuracy": train_accuracy, "Test Accuracy": test_accuracy}.items():
    plt.plot(data)
    plt.title(title)
    plt.show()